In [4]:
!pip install pandas openpyxl rdkit-pypi

ERROR: Could not find a version that satisfies the requirement rdkit-pypi (from versions: none)
ERROR: No matching distribution found for rdkit-pypi


In [5]:
!pip install rdkit

In [8]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Draw
import os

# Leer archivo Excel
df = pd.read_excel('cuasinoides_moldraw.xlsx')

# Crear directorio para guardar imágenes
output_dir = 'estructuras_cuasinoides'
os.makedirs(output_dir, exist_ok=True)

print(f"Procesando {len(df)} compuestos...")

# Listas para seguimiento
exitosos = []
errores = []

for idx, row in df.iterrows():
    smiles = str(row['SMILES']).strip()
    name = str(row['Name']).strip()
    
    print(f"Procesando: {name}")
    
    # Convertir SMILES a molécula
    mol = Chem.MolFromSmiles(smiles)
    
    if mol is not None:
        try:
            # Generar y guardar imagen individual
            img = Draw.MolToImage(mol, size=(500, 400), legend=name)
            
            # Crear nombre de archivo seguro
            safe_name = "".join(c for c in name if c.isalnum() or c in (' ', '-', '_')).rstrip()
            filename = f"{safe_name}.png"
            filepath = os.path.join(output_dir, filename)
            
            img.save(filepath)
            exitosos.append(name)
            print(f"  ✅ Guardado: {filename}")
            
        except Exception as e:
            error_msg = f"Error al guardar {name}: {str(e)}"
            errores.append(error_msg)
            print(f"  ❌ {error_msg}")
    else:
        error_msg = f"SMILES inválido para {name}: {smiles}"
        errores.append(error_msg)
        print(f"  ❌ {error_msg}")

# Resumen
print("\n" + "="*50)
print("RESUMEN DEL PROCESO")
print("="*50)
print(f"Compuestos procesados exitosamente: {len(exitosos)}")
print(f"Errores: {len(errores)}")

if exitosos:
    print(f"\nImágenes guardadas en: {output_dir}/")
    print("\nCompuestos guardados:")
    for nombre in exitosos:
        print(f"  • {nombre}")

if errores:
    print("\nErrores encontrados:")
    for error in errores:
        print(f"  • {error}")

Procesando 24 compuestos...
Procesando: 11-O-trans-pcoumaroyl amarolide
  ✅ Guardado: 11-O-trans-pcoumaroyl amarolide.png
Procesando: Amarolide
  ✅ Guardado: Amarolide.png
Procesando: Castelalin
  ✅ Guardado: Castelalin.png
Procesando: Castelanolide
  ✅ Guardado: Castelanolide.png
Procesando: Castelanone
  ✅ Guardado: Castelanone.png
Procesando: Casteloside A
  ✅ Guardado: Casteloside A.png
Procesando: Casteloside B
  ✅ Guardado: Casteloside B.png
Procesando: Casteloside C
  ✅ Guardado: Casteloside C.png
Procesando: Chaparramarin
  ✅ Guardado: Chaparramarin.png
Procesando: Chaparrin
  ✅ Guardado: Chaparrin.png
Procesando: Chaparrinone
  ✅ Guardado: Chaparrinone.png
Procesando: Chaparrolide
  ✅ Guardado: Chaparrolide.png
Procesando: Glaucarubinone
  ✅ Guardado: Glaucarubinone.png
Procesando: Glaucarubol
  ✅ Guardado: Glaucarubol.png
Procesando: Glaucarubolone
  ✅ Guardado: Glaucarubolone.png
Procesando: Holacantona
  ✅ Guardado: Holacantona.png
Procesando: Peninsularinone
  ✅ Guardado: 

In [10]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Draw
import os

def procesar_compuestos(archivo_excel):
    """Procesa compuestos desde Excel y genera imágenes"""
    
    # Leer datos
    df = pd.read_excel(archivo_excel)
    output_dir = 'estructuras_cuasinoides'
    os.makedirs(output_dir, exist_ok=True)
    
    print(f"📖 Leyendo {len(df)} compuestos de {archivo_excel}")
    
    # Procesar cada compuesto
    compuestos_exitosos = []
    
    for idx, row in df.iterrows():
        smiles = str(row['SMILES']).strip()
        nombre = str(row['Name']).strip()
        
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            print(f"❌ {nombre}: SMILES inválido")
            continue
            
        try:
            # Generar imagen
            img = Draw.MolToImage(mol, size=(500, 400), legend=nombre)
            
            # Nombre de archivo seguro
            nombre_archivo = f"{idx+1:02d}_{nombre}.png".replace(' ', '_')
            ruta_completa = os.path.join(output_dir, nombre_archivo)
            
            # Guardar imagen
            img.save(ruta_completa)
            compuestos_exitosos.append((mol, nombre))
            print(f"✅ {nombre_archivo}")
            
        except Exception as e:
            print(f"❌ Error guardando {nombre}: {e}")
    
    # Generar grid si hay compuestos exitosos
    if compuestos_exitosos:
        generar_grid(compuestos_exitosos, output_dir)
    
    return len(compuestos_exitosos)

def generar_grid(compuestos, output_dir):
    """Genera un grid con todos los compuestos"""
    try:
        mols = [c[0] for c in compuestos]
        nombres = [c[1] for c in compuestos]
        
        print("\n🖼️  Generando grid de todas las estructuras...")
        
        grid_img = Draw.MolsToGridImage(
            mols,
            molsPerRow=3,
            legends=nombres,
            subImgSize=(400, 300)
        )
        
        grid_path = os.path.join(output_dir, "00_GRID_COMPLETO.png")
        grid_img.save(grid_path)
        print(f"✅ Grid guardado: 00_GRID_COMPLETO.png")
        
    except Exception as e:
        print(f"❌ Error generando grid: {e}")

# Ejecutar el procesamiento
if __name__ == "__main__":
    archivo = 'cuasinoides_moldraw.xlsx'
    
    if os.path.exists(archivo):
        exitosos = procesar_compuestos(archivo)
        print(f"\n🎉 Proceso completado! {exitosos} estructuras generadas")
        print(f"📁 Archivos guardados en: estructuras_cuasinoides/")
    else:
        print(f"❌ No se encontró el archivo: {archivo}")

📖 Leyendo 24 compuestos de cuasinoides_moldraw.xlsx
✅ 01_11-O-trans-pcoumaroyl_amarolide.png
✅ 02_Amarolide.png
✅ 03_Castelalin.png
✅ 04_Castelanolide.png
✅ 05_Castelanone.png
✅ 06_Casteloside_A.png
✅ 07_Casteloside_B.png
✅ 08_Casteloside_C.png
✅ 09_Chaparramarin.png
✅ 10_Chaparrin.png
✅ 11_Chaparrinone.png
✅ 12_Chaparrolide.png
✅ 13_Glaucarubinone.png
✅ 14_Glaucarubol.png
✅ 15_Glaucarubolone.png
✅ 16_Holacantona.png
✅ 17_Peninsularinone.png
✅ 18_Polyandrol.png
✅ 19_1-epi-holacanthone.png
✅ 20_15-O-acetyl-glaucarubol.png
✅ 21_15-O-acetyl-Δ4,5_glaucarubol.png
✅ 22_1-epi-5-iso-glaucarubolone.png
✅ 23_1-epi-glaucarubolone.png
✅ 24_Δ4,5_glaucarubol.png

🖼️  Generando grid de todas las estructuras...
❌ Error generando grid: 'SVG' object has no attribute 'save'

🎉 Proceso completado! 24 estructuras generadas
📁 Archivos guardados en: estructuras_cuasinoides/


In [11]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Draw
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import landscape, A4
import tempfile
import math

# Carga rápida y PDF simple
df = pd.read_excel('cuasinoides_moldraw.xlsx')

# Estructuras válidas
estructuras = []
for _, row in df.iterrows():
    mol = Chem.MolFromSmiles(str(row['SMILES']))
    if mol:
        estructuras.append((mol, row['Name']))

# Crear PDF
c = canvas.Canvas("grid_rapido.pdf", pagesize=landscape(A4))
ancho, alto = landscape(A4)

cols = 4
filas = math.ceil(len(estructuras) / cols)

for i, (mol, nombre) in enumerate(estructuras):
    fila = i // cols
    col = i % cols
    
    x = 50 + col * (ancho-100)/cols
    y = alto - 100 - fila * (alto-150)/filas
    
    img = Draw.MolToImage(mol, size=(150, 120))
    temp_file = tempfile.NamedTemporaryFile(suffix='.png', delete=False).name
    img.save(temp_file)
    
    c.drawImage(temp_file, x, y, width=120, height=100)
    c.drawString(x, y-10, f"{i+1}. {nombre}")
    
    os.unlink(temp_file)

c.save()
print(f"✅ PDF rápido creado con {len(estructuras)} estructuras")

✅ PDF rápido creado con 24 estructuras


In [14]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Draw
import os
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import landscape, A4
import tempfile
import math

def crear_pdf_estructuras_grandes(archivo_excel, pdf_nombre="estructuras_grandes.pdf"):
    """
    Versión con estructuras más grandes y limpias
    """
    # Leer datos
    df = pd.read_excel(archivo_excel)
    
    # Procesar estructuras válidas
    compuestos = []
    for idx, row in df.iterrows():
        smiles = str(row['SMILES']).strip()
        mol = Chem.MolFromSmiles(smiles)
        if mol is not None:
            compuestos.append(mol)
    
    if not compuestos:
        print("No hay estructuras válidas para procesar.")
        return
    
    n = len(compuestos)
    print(f"Procesando {n} estructuras...")
    
    # Configuración para estructuras grandes
    c = canvas.Canvas(pdf_nombre, pagesize=landscape(A4))
    ancho, alto = landscape(A4)
    
    # Grid más espaciado para estructuras grandes
    if n <= 4:
        cols = 2
    elif n <= 6:
        cols = 3
    elif n <= 9:
        cols = 3
    elif n <= 12:
        cols = 4
    else:
        cols = 4  # Máximo 4 columnas para mantener tamaño
    
    filas = math.ceil(n / cols)
    
    # Márgenes optimizados para estructuras
    margen = 40
    area_util_x = ancho - 2 * margen
    area_util_y = alto - 2 * margen
    
    celda_ancho = area_util_x / cols
    celda_alto = area_util_y / filas
    
    # Sin título para maximizar espacio
    for i, mol in enumerate(compuestos):
        fila = i // cols
        columna = i % cols
        
        x = margen + columna * celda_ancho
        y = alto - margen - (fila + 1) * celda_alto
        
        try:
            # Tamaño generoso para estructuras
            img_ancho = celda_ancho - 20
            img_alto = celda_alto - 20
            
            # Generar imagen
            img_size = (int(img_ancho), int(img_alto))
            img = Draw.MolToImage(mol, size=img_size)
            temp_path = tempfile.NamedTemporaryFile(suffix='.png', delete=False).name
            img.save(temp_path)
            
            # Centrar estructura en la celda
            c.drawImage(temp_path, 
                       x + 10, 
                       y + 10, 
                       width=img_ancho, 
                       height=img_alto)
            
            os.unlink(temp_path)
            
        except Exception as e:
            print(f"Error con estructura {i+1}: {e}")
    
    c.save()
    print(f"✅ PDF con {n} estructuras creado: {pdf_nombre}")

# Ejecutar
crear_pdf_estructuras_grandes('cuasinoides_moldraw.xlsx')

Procesando 24 estructuras...
✅ PDF con 24 estructuras creado: estructuras_grandes.pdf


In [15]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Draw
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import landscape, A4
import tempfile
import math

# Carga y procesamiento mínimo
df = pd.read_excel('cuasinoides_moldraw.xlsx')

mols = [Chem.MolFromSmiles(str(row['SMILES'])) for _, row in df.iterrows()]
mols = [mol for mol in mols if mol is not None]

print(f"Generando PDF con {len(mols)} estructuras...")

c = canvas.Canvas("estructuras_minimal.pdf", pagesize=landscape(A4))
ancho, alto = landscape(A4)

cols = 4
filas = math.ceil(len(mols) / cols)

for i, mol in enumerate(mols):
    fila = i // cols
    col = i % cols
    
    x = 40 + col * (ancho-80)/cols
    y = alto - 40 - (fila + 1) * (alto-80)/filas
    
    img = Draw.MolToImage(mol, size=(200, 160))
    temp_file = tempfile.NamedTemporaryFile(suffix='.png', delete=False).name
    img.save(temp_file)
    
    c.drawImage(temp_file, x, y, width=150, height=120)
    os.unlink(temp_file)

c.save()
print(f"✅ PDF minimalista creado")

Generando PDF con 24 estructuras...
✅ PDF minimalista creado


In [18]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Draw
import os
import math
from PIL import Image
import tempfile

def crear_png_grid_estructuras(archivo_excel, png_nombre="grid_estructuras.png"):
    """
    Crea un archivo PNG con grid de estructuras químicas
    """
    # Leer datos
    df = pd.read_excel(archivo_excel)
    
    # Procesar estructuras válidas
    compuestos = []
    for idx, row in df.iterrows():
        smiles = str(row['SMILES']).strip()
        mol = Chem.MolFromSmiles(smiles)
        if mol is not None:
            compuestos.append(mol)
    
    if not compuestos:
        print("No hay estructuras válidas para procesar.")
        return
    
    n = len(compuestos)
    print(f"Procesando {n} estructuras válidas...")
    
    # Calcular grid óptimo
    if n <= 4:
        cols = 2
        img_size = (400, 300)
    elif n <= 9:
        cols = 3
        img_size = (300, 250)
    elif n <= 16:
        cols = 4
        img_size = (250, 200)
    elif n <= 25:
        cols = 5
        img_size = (200, 160)
    else:
        cols = 6
        img_size = (180, 140)
    
    # Crear imagen grid - método corregido
    try:
        grid_img = Draw.MolsToGridImage(
            compuestos,
            molsPerRow=cols,
            subImgSize=img_size,
            returnPNG=True  # Forzar PNG
        )
        
        # Guardar PNG
        grid_img.save(png_nombre)
        print(f"🎉 PNG creado exitosamente: {png_nombre}")
        print(f"🔢 Estructuras: {n}")
        print(f"📐 Grid: {cols} × {math.ceil(n / cols)}")
        
    except Exception as e:
        print(f"❌ Error al crear PNG: {e}")
        # Método alternativo si falla
        crear_png_alternativo(compuestos, png_nombre, cols, img_size)

def crear_png_alternativo(mols, png_nombre, cols, img_size):
    """Método alternativo para crear PNG"""
    try:
        # Crear imágenes individuales y luego combinarlas
        images = []
        for mol in mols:
            img = Draw.MolToImage(mol, size=img_size)
            images.append(img)
        
        # Calcular dimensiones del grid
        rows = math.ceil(len(mols) / cols)
        grid_width = cols * img_size[0]
        grid_height = rows * img_size[1]
        
        # Crear imagen vacía
        grid_img = Image.new('RGB', (grid_width, grid_height), 'white')
        
        # Pegar cada imagen en su posición
        for i, img in enumerate(images):
            row = i // cols
            col = i % cols
            x = col * img_size[0]
            y = row * img_size[1]
            grid_img.paste(img, (x, y))
        
        # Guardar
        grid_img.save(png_nombre)
        print(f"✅ PNG creado (método alternativo): {png_nombre}")
        
    except Exception as e:
        print(f"❌ Error en método alternativo: {e}")

# Ejecutar
crear_png_grid_estructuras('cuasinoides_moldraw.xlsx')

Procesando 24 estructuras válidas...
❌ Error al crear PNG: 'SVG' object has no attribute 'save'
✅ PNG creado (método alternativo): grid_estructuras.png


In [19]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Draw
from PIL import Image
import math

def crear_png_confiable(archivo_excel, png_nombre="grid_confiable.png"):
    """
    Versión que solo usa el método manual (más confiable)
    """
    # Leer datos
    df = pd.read_excel(archivo_excel)
    
    # Procesar estructuras
    mols = []
    for _, row in df.iterrows():
        mol = Chem.MolFromSmiles(str(row['SMILES']))
        if mol:
            mols.append(mol)
    
    n = len(mols)
    if n == 0:
        print("No hay estructuras válidas.")
        return
    
    print(f"Creando PNG con {n} estructuras...")
    
    # Configurar grid
    if n <= 4:
        cols = 2
        img_size = (400, 300)
    elif n <= 9:
        cols = 3
        img_size = (300, 250)
    elif n <= 16:
        cols = 4
        img_size = (250, 200)
    else:
        cols = 5
        img_size = (200, 160)
    
    rows = math.ceil(n / cols)
    
    # Crear imagen grid
    grid_width = cols * img_size[0]
    grid_height = rows * img_size[1]
    grid_img = Image.new('RGB', (grid_width, grid_height), 'white')
    
    # Agregar cada estructura
    for i, mol in enumerate(mols):
        img = Draw.MolToImage(mol, size=img_size)
        row = i // cols
        col = i % cols
        x = col * img_size[0]
        y = row * img_size[1]
        grid_img.paste(img, (x, y))
    
    # Guardar
    grid_img.save(png_nombre)
    print(f"✅ PNG creado exitosamente: {png_nombre}")
    print(f"📐 Tamaño: {grid_width} × {grid_height} px")
    print(f"🔢 Estructuras: {n} en grid {cols}×{rows}")

# Ejecutar
crear_png_confiable('cuasinoides_moldraw.xlsx')

Creando PNG con 24 estructuras...
✅ PNG creado exitosamente: grid_confiable.png
📐 Tamaño: 1000 × 800 px
🔢 Estructuras: 24 en grid 5×5
